In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Tue Apr  5 00:21:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   42C    P8    12W / 185W |    653MiB /  8192MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install gym==0.10.5

In [3]:
# Install gym-snake
!pip install -e ./Gym-Snake-master/

Obtaining file:///G:/Git/NUS_MComp/CS5446/cs5446_grp_proj/Gym-Snake-master
  Attempting uninstall: gym-snake
    Found existing installation: gym-snake 0.0.1
    Uninstalling gym-snake-0.0.1:
      Successfully uninstalled gym-snake-0.0.1
  Running setup.py develop for gym-snake


In [4]:
import random, time, gym, sys
import gym.spaces
from gym.wrappers.monitor import Monitor
import itertools
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from collections import namedtuple
import pickle
import time
import uuid
import os
import sys
from collections import deque

OptimizerSpec = namedtuple("OptimizerSpec", ["constructor", "kwargs", "lr_schedule"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import gym_snake
# env=gym.make('snake-v0')

In [6]:
# print(env)

## Set Up Hyperparameters

In [7]:
# Set snake game environment and set up
env='snake-v0'
seed=3
num_steps=5e4
double_q=True


if seed is None:
    seed = random.randint(0, 9999)
print('random seed = {}'.format(seed))
exp_name = 'double-dqn'

logdir = exp_name+ '_' +env+ '_' +time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join('data_dqn', logdir)

def set_global_seeds(i):
    np.random.seed(i)
    random.seed(i)

env = gym.make(env)
set_global_seeds(seed)
env.seed(seed)
torch.manual_seed(seed)
expt_dir = os.path.join(logdir, "gym")
env = Monitor(env, expt_dir, force=True, video_callable=False)
print(env)

random seed = 3
<Monitor<SnakeEnv<snake-v0>>>


G:\Tool\Miniconda\envs\deeplearn_course\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [8]:
# last_obs = env.reset()
# %matplotlib notebook
print(env.observation_space)

None


In [9]:
# print(last_obs.shape)
# print(last_obs)

In [10]:
# # %matplotlib notebook
# env.render()

In [11]:
class DQN(nn.Module):

    def __init__(self, input_size, num_actions):
        super(DQN, self).__init__()
        self.input_size = input_size
        self.pool1  = nn.MaxPool2d(10,10)
        self.dense1 = nn.Linear(input_size, 1024)
        self.dense2 = nn.Linear(1024, 1024)
        self.dense3 = nn.Linear(1024, num_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = self.pool1(x)
        x = x.view(-1, self.input_size)
        x = F.tanh(self.dense1(x))
        x = F.tanh(self.dense2(x))
        out = self.dense3(x)
        return out

## Set Up Exploration Strategy

In [12]:
def linear_interpolation(l, r, alpha):
    return l + alpha * (r - l)

class PiecewiseSchedule(object):

    def __init__(self, endpoints, interpolation=linear_interpolation, outside_value=None):
        """Piecewise schedule.
        
        Parameters
        ----------
        endpoints: [(int, int)]
            list of pairs `(time, value)` meanining that schedule should output
            `value` when `t==time`. All the values for time must be sorted in
            an increasing order. When t is between two times, e.g. `(time_a, value_a)`
            and `(time_b, value_b)`, such that `time_a <= t < time_b` then value outputs
            `interpolation(value_a, value_b, alpha)` where alpha is a fraction of
            time passed between `time_a` and `time_b` for time `t`.
        interpolation: lambda float, float, float: float
            a function that takes value to the left and to the right of t according
            to the `endpoints`. Alpha is the fraction of distance from left endpoint to
            right endpoint that t has covered. See linear_interpolation for example.
        outside_value: float
            if the value is requested outside of all the intervals sepecified in
            `endpoints` this value is returned. If None then AssertionError is
            raised when outside value is requested.
        """
        idxes = [e[0] for e in endpoints]
        assert idxes == sorted(idxes)
        self._interpolation = interpolation
        self._outside_value = outside_value
        self._endpoints      = endpoints

    def value(self, t):
        """See Schedule.value"""
        for (l_t, l), (r_t, r) in zip(self._endpoints[:-1], self._endpoints[1:]):
            if l_t <= t and t < r_t:
                alpha = float(t - l_t) / (r_t - l_t)
                return self._interpolation(l, r, alpha)

        # t does not belong to any of the pieces, so doom.
        assert self._outside_value is not None
        return self._outside_value

exploration_schedule = PiecewiseSchedule([
        (0,   1.00),
        (5e3, 0.03),
        (1e4, 0.01),
    ], outside_value=0.01
)

In [13]:
_endpoints = [
        (0,   1.00),
        (5e3, 0.03),
        (1e4, 0.01),
    ]
for (l_t, l), (r_t, r) in zip(_endpoints[:-1], _endpoints[1:]):
    print(l_t, l, r_t, r)

0 1.0 5000.0 0.03
5000.0 0.03 10000.0 0.01


## Set Up Replay Buffer

In [14]:
# ------------------------------------------------------------------------------
# REPLAY BUFFER
# ------------------------------------------------------------------------------

class BasicBuffer:

  def __init__(self, max_size):
      self.max_size = max_size
      self.buffer = deque(maxlen=max_size)

  def push(self, state, action, reward, next_state, done):
      experience = (state, action, np.array([reward]), next_state, done)
      self.buffer.append(experience)

  def sample(self, batch_size):
      state_batch = []
      action_batch = []
      reward_batch = []
      next_state_batch = []
      done_batch = []

      batch = random.sample(self.buffer, batch_size)

      for experience in batch:
          state, action, reward, next_state, done = experience
          state_batch.append(state)
          action_batch.append(action)
          reward_batch.append(reward)
          next_state_batch.append(next_state)
          done_batch.append(done)

      return (state_batch, action_batch, reward_batch, next_state_batch, done_batch)

  def __len__(self):
      return len(self.buffer)

def get_wrapper_by_name(env, classname):
    currentenv = env
    while True:
        if classname in currentenv.__class__.__name__:
            return currentenv
        elif isinstance(env, gym.Wrapper):
            currentenv = currentenv.env
        else:
            raise ValueError("Couldn't find wrapper named %s"%classname)

## Run Q-Learning Algorithm

## Noted you need to implement *Double DQN* in this assignment.

In [15]:
class QLearner(object):

    def __init__(self,
                 env,
                 q_func,
                 t_func,
                 optimizer_spec,
                 exploration,
                 replay_buffer_size,
                 batch_size,
                 gamma,
                 learning_starts,
                 target_update_freq,
                 grad_norm_clipping,
                 double_q=True,
                 max_steps=2e8,
                 cartpole=True):
        """Run Deep Q-learning algorithm.

        You can specify your own convnet using `q_func`.
        All schedules are w.r.t. total number of steps taken in the environment.

        Parameters
        ----------
        env: gym.Env
            gym environment to train on.
        q_func: function
            Model to use for computing the q function. 
        t_func: function
            Model to use for computing the target q function. 
        optimizer_spec: OptimizerSpec
            Specifying the constructor and kwargs, as well as learning rate schedule
            for the optimizer
        exploration: 
            exploration schedule
        replay_buffer_size: int
            How many memories to store in the replay buffer.
        batch_size: int
            How many transitions to sample each time experience is replayed.
        gamma: float
            Discount Factor
        learning_starts: int
            After how many environment steps to start replaying experiences
        target_update_freq: int
            How many experience replay rounds (not steps!) to perform between
            each update to the target Q network
        grad_norm_clipping: float or None
            If not None gradients' norms are clipped to this value.
        double_q: bool
            If True, use double Q-learning to compute target values. Otherwise, vanilla DQN.
            https://papers.nips.cc/paper/3964-double-q-learning.pdf
        max_steps: int
            Maximum number of training steps. The number of *frames* is 4x this
            quantity (modulo the initial random no-op steps).
        cartpole: bool
            If True, CartPole-v0.
        """
#         print(env.observation_space)
#         print(env.action_space)
#         assert type(env.observation_space) == gym.spaces.Box
#         assert type(env.action_space)      == gym.spaces.Discrete
        self.q_func = q_func
        self.t_func = t_func
        self.max_steps = int(max_steps)
        self.target_update_freq = target_update_freq
        self.optimizer = optimizer_spec
        self.exploration = exploration
        self.batch_size = batch_size
        self.gamma = gamma
        self.learning_starts = learning_starts
        self.double_q = double_q
        self.cartpole = cartpole
        self.env = env
        self.replay_buffer = BasicBuffer(max_size=replay_buffer_size)

        # self.input_shape = self.env.observation_space.shape # should be (4,)
        self.num_actions = self.env.action_space.n

        self.num_param_updates = 0
        self.mean_episode_reward      = -float('nan')
        self.std_episode_reward       = -float('nan')
        self.best_mean_episode_reward = -float('inf') 
        self.steps = []
        self.mean_rewards = []
        self.log_every_n_steps = 200
        self.start_time = time.time()
        self.last_obs = self.env.reset()
        self.t = 0


    def step_env(self):
        """Step the env and store the transition.

        At this point, `self.last_obs` contains the latest observation that was
        recorded from the simulator. Here, your code
        needs to store this observation and outcome (reward, next observation,
        etc.) into the replay buffer while doing one step in the env simulator.

        At the end of this block of code, the simulator should have been
        advanced one step, the replay buffer should contain one more transition,
        and, `self.last_obs` must point to the new latest observation.

        Useful functions you'll need to call:

            self.env.step(action)
            self.replay_buffer.push(self.last_obs, action, reward, next_obs, done)

        This steps the environment forward one step. And:

            obs = self.env.reset()

        This resets the environment if you reached an episode boundary.  Call
        `self.env.reset()` to get a new observation if `done=True`. For CartPole, 
        this is guaranteed to start a new episode as they don't have a
        notion of 'ale.lives' in them.

        Don't forget to include epsilon greedy exploration!  
        """

        # do below steps to collect dataset until replay_buffer_size is reached

        # Exploration
        if random.random() < self.exploration.value(self.t):
            # perform random action 0:UP 1:RIGHT 2:DOWN 3:LEFT
            action = random.randint(0, 3)
        else:
            # perform action that maximizes Q
            # compute Q for all actions
            input_to_be_formatted = torch.Tensor(self.last_obs)
            #print(input_to_be_formatted.size())
            input = torch.zeros(input_to_be_formatted.size(2), input_to_be_formatted.size(0), input_to_be_formatted.size(1))
            #print(input.size())
            input[0, :, :] = input_to_be_formatted[:, :, 0]
            input[1, :, :] = input_to_be_formatted[:, :, 1]
            input[2, :, :] = input_to_be_formatted[:, :, 2]
            input = input.to(device)
            scores = self.q_func(input)
            action = torch.argmax(scores).item()

        # step the environment with action
        next_obs, reward, done, _ = self.env.step(action)

        # record to replay buffer
        self.replay_buffer.push(self.last_obs, action, reward, next_obs, done)

        if done == True:
            print("done")
            self.last_obs = self.env.reset()
        else:
            self.last_obs = next_obs


    def update_model(self):
        """Perform experience replay and train the network.

        This is only done if the replay buffer contains enough samples for us to
        learn something useful -- until then, the model will not be initialized
        and random actions should be taken.  Training consists of four steps:
        
        3.a: Use the replay buffer to sample a batch of transitions. See the
        replay buffer code for function definitions. You may use this function:

          self.replay_buffer.sample(self.batch_size)

        3.b: Compute the total Bellman error in a batch. You can use MSE loss to
        achieve it.
        
        3.c: Perform a gradient step and update the network parameters to reduce
        total_error. 
        
        3.d: Periodically update the target network (self.t_func) every 
        `self.target_update_freq` steps.

        """
        if (self.t > self.learning_starts and \
            len(self.replay_buffer) > self.batch_size):
 
            # sample batch of transitions from replay buffer
            state_batch, action_batch, reward_batch, next_state_batch, done_batch = self.replay_buffer.sample(self.batch_size)

            # q function scores and yt_DoubleDQN
            q_func_scores = torch.zeros(self.batch_size)
            q_func_values = torch.zeros(self.batch_size)
            yt_ddqn = torch.zeros(self.batch_size)

            reward_batch = torch.Tensor(reward_batch)
            next_state_batch = torch.Tensor(next_state_batch)
            state_batch = torch.Tensor(state_batch)
            reward_batch = reward_batch.to(device)
            
            next_state_batch_formatted = torch.zeros(next_state_batch.size(0), next_state_batch.size(3), next_state_batch.size(1), next_state_batch.size(2))
            next_state_batch_formatted[:, 0, :, :] = next_state_batch[:, :, :, 0]
            next_state_batch_formatted[:, 1, :, :] = next_state_batch[:, :, :, 1]
            next_state_batch_formatted[:, 2, :, :] = next_state_batch[:, :, :, 2]
            next_state_batch_formatted = next_state_batch_formatted.to(device)
            next_state_batch_formatted.requires_grad_()
            
            state_batch_formatted = torch.zeros(state_batch.size(0), state_batch.size(3), state_batch.size(1), state_batch.size(2))
            state_batch_formatted[:, 0, :, :] = state_batch[:, :, :, 0]
            state_batch_formatted[:, 1, :, :] = state_batch[:, :, :, 1]
            state_batch_formatted[:, 2, :, :] = state_batch[:, :, :, 2]
            state_batch_formatted = state_batch_formatted.to(device)
            state_batch_formatted.requires_grad_()

            # print(next_state_batch.size())
            #next_state_batch_formatted = 
            best_action_next_state = torch.argmax(self.q_func(next_state_batch_formatted), dim=1)
            # print(best_action_next_state)
            target_scores_next_state = self.t_func(next_state_batch_formatted)
            # print(target_scores_next_state)
            q_func_scores = self.q_func(state_batch_formatted)
            for i in range(self.batch_size):
                yt_ddqn[i] = reward_batch[i] + self.gamma * target_scores_next_state[i][best_action_next_state[i].item()] * (1 - done_batch[i])
                q_func_values[i] = q_func_scores[i][action_batch[i]]

            # compute loss
            # print(q_func_values)
            # print(yt_ddqn)
            criterion = nn.MSELoss()
            loss = criterion(q_func_values, yt_ddqn)
            # print(q_func_values)
            # print(yt_ddqn)
            # print(loss)

            # back propagate and optimizer.step
            self.optimizer.zero_grad()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.q_func.parameters(), 10)
            self.optimizer.step()

            # if self.target_update_freq is reached, update target function
            if self.t % self.target_update_freq == 0:
                # update target function parameters
                self.t_func.load_state_dict(self.q_func.state_dict())

            self.num_param_updates += 1
        self.t += 1

    def log_progress(self):
        episode_rewards = get_wrapper_by_name(self.env, "Monitor").get_episode_rewards()

        if len(episode_rewards) > 0:
            self.mean_episode_reward = np.mean(episode_rewards[-10:])
            self.std_episode_reward  = np.std(episode_rewards[-10:])
        if len(episode_rewards) > 10:
            self.best_mean_episode_reward = \
                max(self.best_mean_episode_reward, self.mean_episode_reward)

        if self.t % self.log_every_n_steps == 0:
            self.steps.append(self.t)
            self.mean_rewards.append(self.mean_episode_reward)
            hours = (time.time() - self.start_time) / (60.*60.)
            print("Steps: ",                 self.t)
            print("Avg_Last_10_Episodes", self.mean_episode_reward)
            print("Std_Last_10_Episodes", self.std_episode_reward)
            print("Best_Avg_10_Episodes", self.best_mean_episode_reward)
            print("Num_Episodes",          len(episode_rewards))
            print("Exploration_Epsilon",   self.exploration.value(self.t))
            print("Elapsed_Time_Hours",    hours)


In [16]:
def dqn_learn(*args, **kwargs):
    alg = QLearner(*args, **kwargs)
    while True:
        # 
        #print("play the game")
        alg.step_env()
        #print("update model")
        alg.update_model()
        #print("log progress")
        alg.log_progress()
        
        if alg.t % 100 == 0:
            print("\nt = {} out of max_steps = {}".format(alg.t, alg.max_steps))
        if alg.t > alg.max_steps:
            print("\nt = {} exceeds max_steps = {}".format(alg.t, alg.max_steps))
            break
    return alg.steps, alg.mean_rewards

In [17]:
n_actions = env.action_space.n

#input dim 150x150x3 --maxpool--> 15x15x3
policy_net = DQN(15*15*3, n_actions).to(device)
target_net = DQN(15*15*3, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=5e-4)

steps, mean_rewards = dqn_learn(
    env=env,
    q_func=policy_net,
    t_func=target_net,
    optimizer_spec=optimizer,
    exploration=exploration_schedule,
    replay_buffer_size=50000,
    batch_size=100,
    gamma=0.99,
    learning_starts=1000,
    target_update_freq=100,
    grad_norm_clipping=10,
    double_q=double_q,
    max_steps=num_steps,
    cartpole=True
    )

# env.close()

done
done
done

t = 100 out of max_steps = 50000
done


G:\Tool\Miniconda\envs\deeplearn_course\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
G:\Tool\Miniconda\envs\deeplearn_course\lib\site-packages\torch\nn\functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


done
done
done
Steps:  200
Avg_Last_10_Episodes -1.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes -inf
Num_Episodes 7
Exploration_Epsilon 0.9612
Elapsed_Time_Hours 0.00012249979707929824

t = 200 out of max_steps = 50000
done
done
done
done
done
done

t = 300 out of max_steps = 50000
done
done
done
done
done
Steps:  400
Avg_Last_10_Episodes -1.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes -1.0
Num_Episodes 18
Exploration_Epsilon 0.9224
Elapsed_Time_Hours 0.00013361089759402804

t = 400 out of max_steps = 50000
done
done
done
done
done

t = 500 out of max_steps = 50000
done
done
done
Steps:  600
Avg_Last_10_Episodes -0.8
Std_Last_10_Episodes 0.4
Best_Avg_10_Episodes -0.8
Num_Episodes 26
Exploration_Epsilon 0.8836
Elapsed_Time_Hours 0.00014527724848853219

t = 600 out of max_steps = 50000
done
done
done

t = 700 out of max_steps = 50000
done
done
done
done
Steps:  800
Avg_Last_10_Episodes -0.8
Std_Last_10_Episodes 0.4
Best_Avg_10_Episodes -0.8
Num_Episodes 33
Exploration_Epsilon 0.844

done
done
done
done
done
done
done

t = 5700 out of max_steps = 50000
done
done
done
done
done
Steps:  5800
Avg_Last_10_Episodes -1.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes -0.6
Num_Episodes 250
Exploration_Epsilon 0.026799999999999997
Elapsed_Time_Hours 4.96795111225711

t = 5800 out of max_steps = 50000
done
done
done
done

t = 5900 out of max_steps = 50000
done
done
done
done
done
done
Steps:  6000
Avg_Last_10_Episodes -0.9
Std_Last_10_Episodes 0.30000000000000004
Best_Avg_10_Episodes -0.6
Num_Episodes 260
Exploration_Epsilon 0.026
Elapsed_Time_Hours 5.175316753983497

t = 6000 out of max_steps = 50000
done
done
done
done
done
done

t = 6100 out of max_steps = 50000
done
done
done
done
done
Steps:  6200
Avg_Last_10_Episodes -1.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes -0.6
Num_Episodes 271
Exploration_Epsilon 0.0252
Elapsed_Time_Hours 5.383684743245443

t = 6200 out of max_steps = 50000
done
done
done
done
done

t = 6300 out of max_steps = 50000
done
done
done
done
done

done
done
done
done
done
done
done
done

t = 11100 out of max_steps = 50000
done
done
done
done
Steps:  11200
Avg_Last_10_Episodes -0.8
Std_Last_10_Episodes 0.4
Best_Avg_10_Episodes -0.6
Num_Episodes 537
Exploration_Epsilon 0.01
Elapsed_Time_Hours 10.572224179771212

t = 11200 out of max_steps = 50000
done
done
done

t = 11300 out of max_steps = 50000
done
done
done
done
done
done
Steps:  11400
Avg_Last_10_Episodes -0.8
Std_Last_10_Episodes 0.4
Best_Avg_10_Episodes -0.6
Num_Episodes 546
Exploration_Epsilon 0.01
Elapsed_Time_Hours 10.780751384827825

t = 11400 out of max_steps = 50000
done
done
done
done
done
done

t = 11500 out of max_steps = 50000
done
done
done
done
done
Steps:  11600
Avg_Last_10_Episodes -0.9
Std_Last_10_Episodes 0.30000000000000004
Best_Avg_10_Episodes -0.6
Num_Episodes 557
Exploration_Epsilon 0.01
Elapsed_Time_Hours 10.987571417159504

t = 11600 out of max_steps = 50000
done
done
done
done
done

t = 11700 out of max_steps = 50000
done
done
done
done
done
Steps:  1

done
done
Steps:  16600
Avg_Last_10_Episodes -0.9
Std_Last_10_Episodes 0.30000000000000004
Best_Avg_10_Episodes -0.6
Num_Episodes 828
Exploration_Epsilon 0.01
Elapsed_Time_Hours 16.203294676674737

t = 16600 out of max_steps = 50000
done
done
done
done
done

t = 16700 out of max_steps = 50000
done
done
done
done
done
done
Steps:  16800
Avg_Last_10_Episodes -1.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes -0.6
Num_Episodes 839
Exploration_Epsilon 0.01
Elapsed_Time_Hours 16.411131149397956

t = 16800 out of max_steps = 50000
done
done
done
done

t = 16900 out of max_steps = 50000
done
done
done
done
done
Steps:  17000
Avg_Last_10_Episodes -1.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes -0.6
Num_Episodes 848
Exploration_Epsilon 0.01
Elapsed_Time_Hours 16.61955926458041

t = 17000 out of max_steps = 50000
done
done
done
done

t = 17100 out of max_steps = 50000
done
done
done
done
Steps:  17200
Avg_Last_10_Episodes -0.9
Std_Last_10_Episodes 0.3
Best_Avg_10_Episodes -0.6
Num_Episodes 85

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(steps, mean_rewards)
plt.xlabel("Iterations")
plt.ylabel("Average Return")